# Programming Assignment: Movie recommendation

- 과제 목표: 뉴럴 네트워크 모델을 설계한 후 모델을 학습하여 각 영화들의 embedding 들을 생성하고, 영화 embedding 을 활용하여 각 사용자에게 맞춤형 영화를 추천

# Notice

<br>

- 과제를 수행하면서 각 task 마다 꼭 주어진 1개의 cell만을 사용할 필요는 없으며, 여러 개의 cell을 추가하여 자유롭게 사용해도 괜찮습니다.
- 과제 수행을 위해 필요한 module이 있다면 추가로 import 해도 괜찮습니다.

# Import Modules

In [1]:
from tqdm import tqdm
import warnings, random
import numpy as np
import pandas as pd
import torch

from itertools import permutations  # For making pairs

warnings.filterwarnings('ignore')

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR

from IPython.display import display


# Data loading

In [2]:
dir = './MovieLens100K/'
df_ratings = pd.read_csv(dir + 'ratings.csv', usecols=['userId', 'movieId', 'rating'])
df_movies = pd.read_csv(dir + 'movies.csv', usecols=['movieId', 'title', 'genres']) # for title-matching

# Preprocessing data

1. 50개 미만의 영화를 본 사용자에 대해서 filter(제외)
2. df_ratings로 부터 각 사용자들이 본 영화를 기록.
3. 사용자 마다 본 영화 목록을 $(movie1, movie2)$, $(movie2, movie1)$ 과 같이 pair로 생성.
    - 즉, 각 사용자 마다 본 영화 목록에 대해 Permutation을 수행
4. 3번 과정이 끝난 후, random을 이용해 각 pair 순서를 무작위로 shuffle.

In [3]:
# item 개수에 따라 filter
df_ratings = df_ratings.groupby('userId').filter(lambda x:len(x)>=50)
df_movies = df_movies[df_movies.movieId.isin(df_ratings.movieId.unique())]

# item encoding
num_items = df_ratings.movieId.nunique()
item_map = dict(zip(df_ratings.movieId.unique(), range(num_items)))
df_ratings['movieId'] = df_ratings['movieId'].map(item_map)
df_movies['movieId']  = df_movies['movieId'].map(item_map)

# user encoding
num_users = df_ratings.userId.nunique()
user_map = dict(zip(df_ratings.userId.unique(), range(num_users)))
df_ratings['userId'] = df_ratings['userId'].map(user_map)

#### Your Code Here
random.seed(42)
np.random.seed(42)

total_pairs = []
for user in df_ratings['userId'].unique():
  df = df_ratings[df_ratings['userId']==user]
  movies = df['movieId'].tolist()
  pairs = list(permutations(movies, 2))
  total_pairs.extend(pairs)

random.shuffle(total_pairs)
total_len = len(total_pairs)

# train/valid/test split
# 6:2:2
train_pairs = total_pairs[:int(total_len*0.6)]
valid_pairs = total_pairs[int(total_len*0.6):int(total_len*0.8)]
test_pairs = total_pairs[int(total_len*0.8):]

# Build and train neural networks for generating movie embeddings

<br>

> ### Problem 1 (40 points)

<br>

- 각 영화 임베딩을 구하기 위해 뉴럴 네트워크 모델을 활용하여 multi-class classification 을 수행

- 설계할 신경망의 기본 구조는 **Input Layer - Hidden(Embedding) Layer - Output Layer**.
    - 강의자료 "Learning Embeddings_part2.pdf"의 slide 7 신경망 구조 이미지 참고

- 현재 Network를 통해 하고자 하는 task는 multi-class classification.
    - 예: $(movie1, movie2)$ 와 같은 입력 데이터와 정답 출력 데이터를 이용해 모델을 학습
        - Input : $movie1$의 one-hot vector
        - Output : $\widehat{movie2}$의 one-hot vector
        - Compute Loss : $\widehat{movie2}$ 와 $movie2$ 간의 Cross-entropy Loss
- 학습이 완료된 이후에 input layer와 hidden(embedding) layer 사이의 weight matrix $W_{in}$를 movie에 대한 embedding vector로 사용이 가능.
> embedding size(# of hidden units)는 100 이하로 두는 것을 권장. <br>
> embedding layer 다음 hidden layer를 더 추가하여 Genre와 같은 추가 정보를 학습에 활용 할 수도 있음 (필수적으로 고려해야할 사항은 아님).

- 설계한 뉴럴 네트워크 모델의 학습이 완료된 후, 학습된 weight matrix $W_{in}$의 행/열벡터를 각 영화에 대한 embedding vector로 간주하여 영화 embedding 들을 구할 수 있음.

**INFO (torch.nn.CrossEntropyLoss와 F.softmax의 관계)**
- https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html

In [4]:
def train_ep(net, ld, opt, crit, dev):
    net.train()
    tot = 0.0

    it = iter(ld)
    while True:
        batch = next(it, None)
        if batch is None:
            break

        x, y = batch
        x = x.to(dev)
        y = y.to(dev)

        opt.zero_grad()
        logit = net(x)
        loss = crit(logit, y)
        loss.backward()
        max_grad_norm = 5.0
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_grad_norm)
        opt.step()

        tot += loss.item() * x.size(0)
    loss_val = tot / len(ld.dataset)
    return loss_val


@torch.no_grad()
def eval_ep(net, ld, crit, dev):
    net.eval()
    tot = 0.0

    it = iter(ld)
    while True:
        batch = next(it, None)
        if batch is None:
            break

        x, y = batch
        x = x.to(dev)
        y = y.to(dev)

        logit = net(x)
        loss = crit(logit, y)
        tot += loss.item() * x.size(0)
    final_loss = tot / len(ld.dataset)
    return final_loss

class PairSet(Dataset):
    def __init__(self, pairs):
        self.p = pairs

    def __len__(self):
        return len(self.p)

    def __getitem__(self, i):
        a, b = self.p[i]
        return torch.tensor(a, dtype=torch.long), torch.tensor(b, dtype=torch.long)

class Net(nn.Module):
    def __init__(self, n, d):
        super().__init__()
        self.emb = nn.Embedding(n, d)
        h = 128
        self.l1 = nn.Linear(d, h)
        self.dp = nn.Dropout(0.05)
        self.l2 = nn.Linear(h, d)
        self.ln = nn.LayerNorm(d)

        nn.init.xavier_uniform_(self.emb.weight)
        nn.init.xavier_uniform_(self.l1.weight)
        nn.init.zeros_(self.l1.bias)
        nn.init.xavier_uniform_(self.l2.weight)
        nn.init.zeros_(self.l2.bias)

    def forward(self, x):
        e = self.emb(x)
        h = F.relu(self.l1(e))
        h = self.dp(h)
        h = self.l2(h)
        h = self.ln(h + e)
        logit = torch.matmul(h, self.emb.weight.t())
        return logit

In [5]:
# embedding dim, batch size, learning rate
dim, bs, lr = 64, 512, 5e-4

# epoch 수 (필요시 조정 가능)
epochs = 2

if torch.cuda.is_available():
    dev = torch.device("cuda")
else:
    dev = torch.device("cpu")

n_movie = int(num_items)


va_ds = PairSet(valid_pairs)
tr_ds = PairSet(train_pairs)
print("train:", len(tr_ds), ", valid:", len(va_ds))

tr_ld = DataLoader(tr_ds, batch_size=bs, shuffle=True)
va_ld = DataLoader(va_ds, batch_size=bs, shuffle=False)

net = Net(n_movie, dim).to(dev)
opt = torch.optim.AdamW(
    net.parameters(),
    lr=lr,
    weight_decay=1e-5,
)

crit = nn.CrossEntropyLoss()
sch = StepLR(opt, step_size=10, gamma=0.7)

ep = 1
while ep <= epochs:
    tr = train_ep(net, tr_ld, opt, crit, dev)
    va = eval_ep(net, va_ld, crit, dev)
    sch.step()

    print(f"[epoch : {ep}/{epochs}] "
          f"train-Loss: {tr:.4f}, valid-Loss: {va:.4f} ")
    ep += 1


with torch.no_grad():
    emb_w = net.emb.weight
    movie_embeddings = emb_w.detach().cpu()

print("movie embeddings shape:", movie_embeddings.shape)

train: 36339331 , valid: 12113110
[epoch : 1/2] train-Loss: 8.2273, valid-Loss: 8.1864 
[epoch : 2/2] train-Loss: 8.1815, valid-Loss: 8.1747 
movie embeddings shape: torch.Size([9633, 64])


# Recommend customized movies to user

<br>

> ### Problem 2 (30 points)

<br>

- 임의의 한명의 사용자에 대하여 해당 사용자가 봤던 영화 n개에 대해 **통합된 embedding vector**를 생성.
    - n개의 embedding vector들에 대해, element-wise한 계산을 통해 통합된 하나의 embedding vector를 생성.
    - 이 embedding vector는 해당 사용자의 전반적인 영화 시청 성향을 나타내는 embedding vector로 간주할 수 있음.
    - 즉, **사용자 1명 당 1개의 embedding vector**를 가짐.
- 통합된 embedding vector와 학습된 weight matrix $W_{in}$의 모든 영화 embedding vector들 간의 유사도를 계산.
- 그 중 유사도가 높은 (top n) 영화들을 선정, 사용자에게 추천.
    > Recommended format : MovieId, Title, Genre, Similarity 가 포함된 형식

In [12]:
# User–Item 기반 영화 추천

# uid = userid -> 사용자 변경 가능
uid = 0
seen = df_ratings[df_ratings["userId"] == uid]["movieId"].unique()
print("Target:", uid, ", Seen Count:", len(seen))

user_emb = movie_embeddings[seen].mean(0, keepdim=True)
norm_p = 2
norm_dim = 1
user_emb = F.normalize(user_emb, p=norm_p, dim=norm_dim)
emb_norm = F.normalize(movie_embeddings, p=norm_p, dim=norm_dim)

sim = (emb_norm @ user_emb.T).squeeze()

sim[seen] = -1e9

# top N의미 -> 사용자 변경 가능
N = 10
top_scores, top_ids = torch.topk(sim, N)

recs = []
i = 0
while i < N:
    mid = int(top_ids[i])
    score = float(top_scores[i])
    info = df_movies[df_movies["movieId"] == mid].iloc[0]

    recs.append({
        "MovieId": mid,
        "Title": info["title"],
        "Genre": info["genres"],
        "Similarity": score
    })
    i += 1

df_rec = pd.DataFrame(recs)
print("[ Recommendation Result ]")
display(df_rec)


# Item–Item 유사도 계산
mat = emb_norm @ emb_norm.T
idx = torch.arange(mat.size(0))
mat[idx, idx] = -1


# Item–Item 유사도 검증 -> 0.8 이상 확인
# 중복 제외 -> EX) k가 80이면 40개의 행 출력
K = 20
top_vals, flat_idx = torch.topk(mat.reshape(-1), K)

pairs = []
seen_pair = set()
n = movie_embeddings.size(0)

j = 0
while j < K:
    f = int(flat_idx[j])
    a, b = divmod(f, n)

    key = (min(a, b), max(a, b))
    if key not in seen_pair:
        seen_pair.add(key)

        m1 = df_movies[df_movies["movieId"] == a].iloc[0]
        m2 = df_movies[df_movies["movieId"] == b].iloc[0]

        pairs.append({
            "Movie_1": m1["title"],
            "Genre_1": m1["genres"],
            "Movie_2": m2["title"],
            "Genre_2": m2["genres"],
            "Similarity": float(top_vals[j])
        })

    j += 1

df_pairs = pd.DataFrame(pairs)
# print("[ Movie–Movie Similarities ]")
# display(df_pairs)

Target: 0 , Seen Count: 232
[ Recommendation Result ]


,MovieId,Title,Genre,Similarity
0,303,Mars Attacks! (1996),Action|Comedy|Sci-Fi,0.702085
1,1508,My Cousin Vinny (1992),Comedy,0.678713
2,1063,Star Trek: First Contact (1996),Action|Adventure|Sci-Fi|Thriller,0.665740
3,1024,Die Hard (1988),Action|Crime|Thriller,0.645417
4,909,Aliens (1986),Action|Adventure|Horror|Sci-Fi,0.633648
5,344,"Sixth Sense, The (1999)",Drama|Horror|Mystery,0.631443
6,911,"Fifth Element, The (1997)",Action|Adventure|Comedy|Sci-Fi,0.612759
7,1328,"Nightmare Before Christmas, The (1993)",Animation|Children|Fantasy|Musical,0.604592
8,408,GoldenEye (1995),Action|Adventure|Thriller,0.603321
9,233,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,0.602011
